In [8]:
! pip install pyspark
! pip install findspark
! pip install folium
! pip install geopy
! pip install scikit-learn

In [46]:
import os
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, min, max, countDistinct, desc, explode, split, col, isnull, sum, isnan, when, count, array, reverse, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from scipy.stats import kurtosis, skew
import math
import folium
from geopy.geocoders import Nominatim
from itertools import chain

In [2]:
findspark.init()

In [3]:
spark_url = 'local'
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Project')\
        .getOrCreate()
spark.conf.set("spark.sql.csv.parser.multiLine", "true")

In [26]:
path = 'bangkok_traffy.csv'
df = spark.read.option("multiLine", "true").csv(path, header=True, inferSchema=True)
df.show(5)

+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------+--------------------+--------------------+--------------+----+------------+--------------------+
|  ticket_id|               type|        organization|             comment|               photo|         photo_after|            coords|             address|subdistrict|district|            province|           timestamp|         state|star|count_reopen|       last_activity|
+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------+--------------------+--------------------+--------------+----+------------+--------------------+
|2021-9LHDM6|                 {}|                null|            ไม่มีภาพ|https://storage.g...|                null|100.48661,13.79386|1867 จรัญสนิทวงศ์...|    บางพลัด| บางพล

In [27]:
null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show()

+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+
|ticket_id|type|organization|comment|photo|photo_after|coords|address|subdistrict|district|province|timestamp|state|  star|count_reopen|last_activity|
+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+
|     2441|1570|        2659|   3959| 2114|      85681|  2044|   4486|       2114|    2117|    2423|     2258| 2051|164873|      120071|         2574|
+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+



In [28]:
# Filter the DataFrame to include only rows with null values in the "column_name" column
ticket_id_null_df = df.filter(df.ticket_id.isNull())
coords_null_df = df.filter(df.coords.isNull())
address_null_df = df.filter(df.address.isNull())

def check_first_null(filtered_df):
    # Check if the filtered DataFrame is empty
    if filtered_df.count() == 0:
        return f"No null values in {filtered_df}."
    else:
        first_row = filtered_df.head()
        return first_row

print(check_first_null(ticket_id_null_df), "\n")
print(check_first_null(coords_null_df), "\n")
print(check_first_null(address_null_df), "\n")

Row(ticket_id=None, type='{ถนน}', organization='สำนักงาน ป.ป.ท.,เขตจอมทอง,ฝ่ายเทศกิจ เขตจอมทอง,ผอ.เขตจอมทอง (นายณัฐพงษ์),กลุ่มกรุงธนเหนือ (รองปลัดฯ เฉลิมพล)', comment=None, photo='https://storage.googleapis.com/traffy_public_bucket/TeamChadChart/corruption_photo2.png', photo_after='https://storage.googleapis.com/traffy_public_bucket/attachment/2022-06/e9596093de70ae8abacd6574f26a2d0f4466fe9f.jpg', coords='100.45568,13.69103', address=None, subdistrict='บางขุนเทียน', district='จอมทอง', province='กรุงเทพมหานคร', timestamp='2022-06-09 23:34:34.98044+00', state='เสร็จสิ้น', star='5', count_reopen=None, last_activity='2022-06-10 11:02:34.607728+00') 

Row(ticket_id='2022-7DABXT', type='{สะพาน}', organization=None, comment='"เคยดีใจมีสายสีน้ำเงินสถานี""แยกไฟฉาย""', photo=None, photo_after=None, coords=None, address=None, subdistrict=None, district=None, province=None, timestamp=None, state=None, star=None, count_reopen=None, last_activity=None) 

Row(ticket_id='2022-7DABXT', type='{สะพาน}', 

From above cells give us the first row with null value from each column selected (ticket_id, coords, address) to tell some relationship of those null value.
1. The ticket_id is null when the state='เสร็จสิ้น', it's mean we cannot drop this column significantly.
2. The address has null value 2 times more than coords. In the first null row we can see both of them are null. So it might tell that if no coords, no address too and not vice versa. We'll check in next step. 

In [30]:
sub = ['coords', 'address']
df = df.dropna(how='all', subset=sub)
sub_null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in sub])
sub_null_counts.show()

+------+-------+
|coords|address|
+------+-------+
|     1|   2443|
+------+-------+



As we can see, the null coords most dissapear. So, we don't need to map any address to coords and we can also drop all remained null column.

In [31]:
df = df.dropna(how='all', subset=['coords'])
df = df.dropna(how='all', subset=['address'])

From this step, we'll use df_use to be a data for ML

In [58]:
# change state to boolean 
df_use = df.withColumn('state', when(df.state == 'เสร็จสิ้น', 1).otherwise(0))

# change coords to pair and swap them into format [latitude, longtitude]
flatten = udf(lambda x: list(chain.from_iterable(x)), ArrayType(StringType()))
df_use = df_use.withColumn('coords', array(reverse(split(df.coords, ','))))
df_use = df_use.withColumn('coords', flatten('coords'))
df_use.show(5)

+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+--------------------+--------------------+-----+----+------------+--------------------+
|  ticket_id|               type|        organization|             comment|               photo|         photo_after|              coords|             address|subdistrict|district|            province|           timestamp|state|star|count_reopen|       last_activity|
+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+--------------------+--------------------+-----+----+------------+--------------------+
|2021-9LHDM6|                 {}|                null|            ไม่มีภาพ|https://storage.g...|                null|[13.79386, 100.48...|1867 จรัญสนิทวงศ์...|    บางพลัด| บางพลัด|       กรุงเทพมห

In [60]:
df_use.printSchema()

root
 |-- ticket_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- photo: string (nullable = true)
 |-- photo_after: string (nullable = true)
 |-- coords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- address: string (nullable = true)
 |-- subdistrict: string (nullable = true)
 |-- district: string (nullable = true)
 |-- province: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- state: integer (nullable = false)
 |-- star: string (nullable = true)
 |-- count_reopen: string (nullable = true)
 |-- last_activity: string (nullable = true)

